In [ ]:
# Importing necessary libraries
import uvicorn
import pickle
from fastapi import FastAPI,Request
from pydantic import BaseModel
import pickle
import pandas as pd
import re
import logging
from openai import AzureOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.text_splitter import  RecursiveCharacterTextSplitter,  CharacterTextSplitter

from langchain_community.document_loaders import JSONLoader, DirectoryLoader
from langchain_community.document_loaders import UnstructuredExcelLoader,  UnstructuredPowerPointLoader, Docx2txtLoader, PyPDFLoader,  UnstructuredWordDocumentLoader, TextLoader, UnstructuredFileLoader

from langchain.chains import RetrievalQA

from langchain.retrievers.merger_retriever import MergerRetriever
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain_experimental.text_splitter import SemanticChunker
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,
    SearchField,  
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration
) 

logger = logging.getLogger(__name__)

# Defining a class for Tariff code prediction
class Data(BaseModel):

    EmailText: str

#Initiating the FastAPI
app = FastAPI(
    title="Email Category Classification",
    description="A simple API that is used to classify the category of Email ",
    version="0.1",
    debug=True
)
#Declaring the variables and environment
open_ai_type = "azure"
open_ai_api_version = "2023-05-15"
open_ai_api_key = "open_ai_api_key"
open_ai_api_key = open_ai_api_key 
open_ai_endpoint = "open_ai_endpoint" # base
embeddings_model = "janus-text-embedding-ada-002"
gpt_deployment_4 = "janus-gpt-4"
gpt_deployment_35 = "janus-gpt-35-turbo"
gpt_deployment_35_16="janus-gpt-35-turbo-16k"
embeddings_model = "janus-text-embedding-ada-002"
vector_store_endpoint = "vector_store_endpoint"
vector_store_key = "vector_store_key"
vector_index_name = "vector_index_name"

#Initiating the LLM, Embedding and Azure AI Search
deployment_name = gpt_deployment_35
llm_model = AzureChatOpenAI(deployment_name=deployment_name, openai_api_key =open_ai_api_key, azure_endpoint = open_ai_endpoint, openai_api_version = open_ai_api_version, temperature=0) 

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embeddings_model,
    openai_api_version=open_ai_api_version,
    azure_endpoint = open_ai_endpoint,
    openai_api_key = open_ai_api_key
)

vector_store = AzureSearch(
    azure_search_endpoint=vector_store_endpoint ,
    azure_search_key=vector_store_key,
    index_name=vector_index_name,
    embedding_function=embeddings.embed_query
)

In [ ]:
loader = TextLoader(r"your_path", encoding="utf-8")
documents = loader.load()

In [ ]:
documents

In [ ]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)   
    num_tokens = len(encoding.encode(string))
    return num_tokens, encoding

print(num_tokens_from_string(documents[0].page_content, "cl100k_base"))


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(documents)

In [ ]:
texts

In [ ]:
document1 = texts[0:100] 
document2 = texts[100:200]
document3 = texts[200:300]
document4 = texts[300:400]
document5 = texts[400:500] 
document6 = texts[500:600]
document7 = texts[600:700]
document8 = texts[700:800]
document9 = texts[800:900]
document10 = texts[900:1000]
document11 = texts[1000:1100] 
document12 = texts[1100:1200]
document13 = texts[1200:1300]
document14 = texts[1300:1400]
document15 = texts[1400:1500] 
document16 = texts[1500:1600]
document17 = texts[1600:1700]
document18 = texts[1700:1800]
document19 = texts[1800:1900]
document20 = texts[1900:2000]
document21 = texts[2000:2100] 
document22 = texts[2100:2200]
document23 = texts[2200:2300]
document24 = texts[2300:2400]
document25 = texts[2400:2500] 
document26 = texts[2500:2600]
document27 = texts[2600:2700]
document28 = texts[2700:2800]
document29 = texts[2800:2900]
document30 = texts[2900:]

In [ ]:
len(document1)+len(document2)+len(document3)+len(document4)+len(document5)+len(document6)+len(document7)+len(document8)+len(document9)+len(document10)+len(document11)+len(document12)+len(document13)+len(document14)+len(document15)+len(document16)+len(document17)+len(document18)+len(document19)+len(document20)+len(document21)+len(document22)+len(document23)+len(document24)+len(document25)+len(document26)+len(document27)+len(document28)+len(document29)+len(document30)

In [ ]:
texts

In [ ]:
vector_store.add_documents(documents=texts)